https://www.thepythoncode.com/article/pretraining-bert-huggingface-transformers-in-python

In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"]="0"

In [2]:
from datasets import *
from transformers import *
from tokenizers import *
import os
import json

/home/john_zhang/.conda/envs/eeg_nlp/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2023-06-01 00:29:56.239504: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-06-01 00:29:56.772099: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/lib:/usr/local/cuda/lib:/usr/local/cuda/lib64:
2023-06-01 00:29:56.772157: W tensorflow/compiler/xla/stream_executor/pla

In [6]:
import pickle
# with open('sam_dataset.pkl', 'wb') as f:
#     pickle.dump(train_data, f)
with open('../sam_dataset.pkl', 'rb') as f:
    train_data = pickle.load(f)

In [ ]:
def trunc(data_array):
    truncated_sentence = []
    for i in range(len(data_array)):
        for j in range(len(data_array[i])-1):
            tmp = data_array[i][j] + data_array[i][j+1]
            # print(len(train_data[i][j]),len(train_data[i][j+1]))
            # print(len(tmp))
            truncated_sentence.append(tmp)
    return truncated_sentence
   

In [9]:
 # Path to the output file
train_data1 = train_data[0:int(len(train_data)*0.8)]
tmp = train_data[int(len(train_data)*0.8):]
test_data1 = tmp[0:int(len(tmp)*0.5)]

val_data1 = tmp[int(len(tmp)*0.5):]
trunc_train = trunc(train_data1)
file_path = "train2.txt" 
with open(file_path, "w") as file:
    for row in trunc_train:
        # Convert each element in the row to a string and join them with a delimiter
        line = "".join(str(elem) for elem in row)
        file.write(line + "\n")
print(len(train_data1))
print(len(trunc_train))
print("="*20)
file_path = "test2.txt" 
trunc_test = trunc(test_data1)
with open(file_path, "w") as file:
    for row in trunc_test:
        # Convert each element in the row to a string and join them with a delimiter
        line = "".join(str(elem) for elem in row)
        file.write(line + "\n")
print(len(test_data1))
print(len(trunc_test))
print("="*20)
file_path = "val2.txt" 
trunc_val = trunc(val_data1)

with open(file_path, "w") as file:
    for row in trunc_val:
        # Convert each element in the row to a string and join them with a delimiter
        line = "".join(str(elem) for elem in row)
        file.write(line + "\n")
print(len(val_data1))
print(len(trunc_val))
print("="*20)

2052
108234
256
13236
257
13320


In [3]:
files = ["data/train2.txt", "data/test2.txt", "data/val2.txt"] # train3.txt, etc.
dataset = load_dataset("text", data_files=files, split="train")

Found cached dataset text (/home/john_zhang/.cache/huggingface/datasets/text/default-f85a1b490e1abc0f/0.0.0/cb1e9bd71a82ad27976be3b12b407850fe2837d80c22c5e03a28949843a8ace2)


In [4]:
d = dataset.train_test_split(test_size=0.1)

In [5]:
len(d['train']['text'][0])/2

3339.0

In [6]:
special_tokens = [
  "[PAD]", "[UNK]", "[CLS]", "[SEP]", "[MASK]", "<S>", "<T>"
]
# if you want to train the tokenizer on both sets
# files = ["train.txt", "test.txt"]
# training the tokenizer on the training set
files = ["output.txt"]
# 30,522 vocab is BERT's default vocab size, feel free to tweak
vocab_size = 30_522
# maximum sequence length, lowering will result to faster training (when increasing batch size)
max_length = 512
# whether to truncate
truncate_longer_samples = True

In [6]:
# training tokenizer
# tokenizer = BertWordPieceTokenizer()
# # train the tokenizer
# tokenizer.train(files=files, vocab_size=vocab_size, special_tokens=special_tokens)
# # enable truncation up to the maximum 512 tokens
# tokenizer.enable_truncation(max_length=max_length)

In [7]:
# training tokenizer

# model_path = "pretrained-bert"
# if not os.path.isdir(model_path):
#   os.mkdir(model_path)
# # save the tokenizer  
# tokenizer.save_model(model_path)
# # dumping some of the tokenizer config to config file, 
# # including special tokens, whether to lower case and the maximum sequence length
# with open(os.path.join(model_path, "config.json"), "w") as f:
#   tokenizer_cfg = {
#       "do_lower_case": True,
#       "unk_token": "[UNK]",
#       "sep_token": "[SEP]",
#       "pad_token": "[PAD]",
#       "cls_token": "[CLS]",
#       "mask_token": "[MASK]",
#       "model_max_length": max_length,
#       "max_len": max_length,
#   }
#   json.dump(tokenizer_cfg, f)

In [ ]:
# model_path = "pretrained-bert"

# tokenizer = BertTokenizerFast.from_pretrained(model_path)

In [9]:
# # bpe tokenizer
# # from transformers import RobertaTokenizerFast
# # tokenizer_folder = "./bpe"

# # tokenizer = RobertaTokenizerFast.from_pretrained(tokenizer_folder, return_special_tokens_mask=True, max_length=512)  
# # bpe tokenizer
# from transformers import RobertaTokenizerFast
# from tokenizers import Tokenizer
# from tokenizers.models import BPE
# from tokenizers.pre_tokenizers import Whitespace

# tokenizer = Tokenizer(BPE())
# from tokenizers.trainers import BpeTrainer

# tokenizer.pre_tokenizer = Whitespace()
# trainer = BpeTrainer(special_tokens=["[UNK]", "[CLS]", "[SEP]", "[PAD]", "[MASK]"])
# tokenizer.train(files=["output.txt"], trainer=trainer)


In [8]:


# tokenizer.save("bpev2.tokenizer.json", pretty=True)

In [7]:
from transformers import PreTrainedTokenizerFast
from tokenizers import Tokenizer
tokenizer = Tokenizer.from_file('bpev2.tokenizer.json')
fast_tokenizer = PreTrainedTokenizerFast(tokenizer_object=tokenizer)
tokenizer = fast_tokenizer
tokenizer.pad_token = "[PAD]"
tokenizer.mask_token = "[MASK]"


In [8]:
def encode_with_truncation(examples):
  """Mapping function to tokenize the sentences passed with truncation"""
  return tokenizer(examples["text"], truncation=True, padding="max_length",
                   max_length=max_length, return_special_tokens_mask=True)

def encode_without_truncation(examples):
  """Mapping function to tokenize the sentences passed without truncation"""
  return tokenizer(examples["text"], return_special_tokens_mask=True)

# the encode function will depend on the truncate_longer_samples variable
encode = encode_with_truncation if truncate_longer_samples else encode_without_truncation
# tokenizing the train dataset
train_dataset = d["train"].map(encode, batched=True)
# tokenizing the testing dataset
test_dataset = d["test"].map(encode, batched=True)
if truncate_longer_samples:
  # remove other columns and set input_ids and attention_mask as PyTorch tensors
  train_dataset.set_format(type="torch", columns=["input_ids", "attention_mask"])
  test_dataset.set_format(type="torch", columns=["input_ids", "attention_mask"])
else:
  # remove other columns, and remain them as Python lists
  test_dataset.set_format(columns=["input_ids", "attention_mask", "special_tokens_mask"])
  train_dataset.set_format(columns=["input_ids", "attention_mask", "special_tokens_mask"])

In [9]:
(test_dataset['input_ids'][0])

tensor([  266,   401,  3825,  7124,  3391,   115,  1410,  2722,   241,   480,
          285,  2958,  3255,   777,   187,  1989,   818,   627,  7185, 13462,
         1791,   327,  4948,  1604,   201,   997,  1049,  1206,   224,   290,
           73,   558,   883,    59,   287,  3510,   239,   491, 11192, 12528,
          730,   213, 10145,  6656,   440, 10648,  4657,  3607,   799,  4288,
        14950,    42, 16986,  2563,   295,   632,   390,   199,  7639,  4259,
         4971,  2148,   373,  7250,   146,  1071,   217,  1129,  4365,  1145,
          389,    46,  5131,    51,  1558,   309, 13945,   657,  2821,   299,
         1798,   229,   408,   252,  1575,  8697,  2059,    18,   327,  2516,
          389, 17804,  9430,   135, 18923,  1311,   219,   308,   485, 10935,
           85,  1272,  2641,    52,   218,   187,   330,   287,  2010,   126,
         6372,   217,   206,  4769,   679,  1164,    28, 19408,  6676,   312,
         2604, 10687,  2045,   913,   113,  1916,   225,   402, 

In [47]:
from itertools import chain
# Main data processing function that will concatenate all texts from our dataset and generate chunks of
# max_seq_length.
# grabbed from: https://github.com/huggingface/transformers/blob/main/examples/pytorch/language-modeling/run_mlm.py
def group_texts(examples):
    # Concatenate all texts.
    concatenated_examples = {k: list(chain(*examples[k])) for k in examples.keys()}
    total_length = len(concatenated_examples[list(examples.keys())[0]])
    # We drop the small remainder, we could add padding if the model supported it instead of this drop, you can
    # customize this part to your needs.
    if total_length >= max_length:
        total_length = (total_length // max_length) * max_length
    # Split by chunks of max_len.
    result = {
        k: [t[i : i + max_length] for i in range(0, total_length, max_length)]
        for k, t in concatenated_examples.items()
    }
    return result

if not truncate_longer_samples:
  train_dataset = train_dataset.map(group_texts, batched=True,
                                    desc=f"Grouping texts in chunks of {max_length}")
  test_dataset = test_dataset.map(group_texts, batched=True,
                                  desc=f"Grouping texts in chunks of {max_length}")
  # convert them from lists to torch tensors
  train_dataset.set_format("torch")
  test_dataset.set_format("torch")

In [10]:
len(train_dataset), len(test_dataset)


(121311, 13479)

In [11]:
# DEFAULT

# initialize the model with the config
model_config = BertConfig(vocab_size=vocab_size, max_position_embeddings=max_length)
model = BertForMaskedLM(config=model_config)

Generate config GenerationConfig {
  "_from_model_config": true,
  "pad_token_id": 0,
  "transformers_version": "4.28.0.dev0"
}



In [11]:
model_config = BertConfig(vocab_size=vocab_size, 
                          max_position_embeddings=max_length,
                          num_attention_heads=1,
                          num_encoder_layers=3,
                          type_vocab_size=1,
                          hidden_size = 250,
                          intermediate_size = 307)
model = BertForMaskedLM(config=model_config)

Generate config GenerationConfig {
  "_from_model_config": true,
  "pad_token_id": 0,
  "transformers_version": "4.28.0.dev0"
}



In [12]:
data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer, mlm=True, mlm_probability=0.2
)

In [13]:
import wandb
%env WANDB_PROJECT=customBert

wandb.login()


Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


env: WANDB_PROJECT=customBert
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


wandb: Currently logged in as: jagohion (ai_essay_gem). Use `wandb login --relogin` to force relogin


True

In [14]:
model_path = "pretrained-bert"
training_args = TrainingArguments(
    output_dir=model_path,          # output directory to where save model checkpoint
    evaluation_strategy="steps",    # evaluate each `logging_steps` steps
    overwrite_output_dir=True,      
    num_train_epochs=800,            # number of training epochs
    per_device_train_batch_size=2, # the training batch size, put it as high as your GPU memory fits
    gradient_accumulation_steps=8,  # accumulating the gradients before updating the weights
    per_device_eval_batch_size=64,  # evaluation batch size
    logging_steps=100,             # evaluate, log and save model checkpoints every 1000 step
    save_steps=1000,
    report_to="wandb",
    run_name="bert_v1",
    load_best_model_at_end=True,  # whether to load the best model (in terms of loss) at the end of training
    save_total_limit=  3         # whether you don't have much space so you let only 3 model weights saved in the disk
)

using `logging_steps` to initialize `eval_steps` to 100
PyTorch: setting up devices


In [15]:
trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
)

In [16]:
trainer.train()


The following columns in the training set don't have a corresponding argument in `BertForMaskedLM.forward` and have been ignored: special_tokens_mask, text. If special_tokens_mask, text are not expected by `BertForMaskedLM.forward`,  you can safely ignore this message.
/home/john_zhang/.conda/envs/eeg_nlp/lib/python3.9/site-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 121311
  Num Epochs = 800
  Instantaneous batch size per device = 2
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 8
  Total optimization steps = 6065600
  Number of trainable parameters = 109514298
Automatic Weights & Biases logging enabled, to disable set os.environ["WANDB_DISABLED"] = "tru

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


You're using a PreTrainedTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Step,Training Loss,Validation Loss


The following columns in the evaluation set don't have a corresponding argument in `BertForMaskedLM.forward` and have been ignored: special_tokens_mask, text. If special_tokens_mask, text are not expected by `BertForMaskedLM.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 13479
  Batch size = 64
***** Running Evaluation *****
  Num examples = 13479
  Batch size = 64


In [ ]:

wandb.finish()

eval/loss,█▇▅▄▄▃▃▃▃▃▃▃▃▃▃▃▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
eval/runtime,▂▁▂▄▅▅▅▆▆▆▆▆▆▆▇▇███████████▇█▇▇▇█▇▇▇▇▇▇▇
eval/samples_per_second,▇█▇▅▄▄▄▃▃▃▃▃▃▂▂▂▁▁▁▁▁▁▁▁▁▁▁▂▁▂▂▂▁▂▂▂▂▂▂▂
eval/steps_per_second,▇█▇▅▄▄▃▃▃▃▃▃▃▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▂▂▂▁▂▂▂▂▂▂▂
train/epoch,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
train/global_step,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
train/learning_rate,████▇▇▇▇▇▆▆▆▆▆▆▅▅▅▅▅▄▄▄▄▄▄▃▃▃▃▃▂▂▂▂▂▂▁▁▁
train/loss,█▇▅▄▃▃▃▃▃▃▃▃▃▃▂▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
train/total_flos,▁
train/train_loss,▁
train/train_runtime,▁


: 

In [20]:
!nvidia-smi

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
Fri May 12 17:01:22 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 495.46       Driver Version: 495.46       CUDA Version: 11.5     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA GeForce ...  Off  | 00000000:01:00.0 Off |                  N/A |
| 48